In [1]:
import tensorflow.keras as keras

In [3]:
keras.__version__

'2.2.4-tf'

In [ ]:
regs = [0, 0]

In [9]:
# gmf
# user, item을 동일한 size의 latent space로.. embedding..후 element wise product..
user_input = keras.layers.Input(shape = (1,), dtype = "int32", name = "user_input")
item_input = keras.layers.Input(shape = (1,), dtype = "int32", name = "item_input")


user_embedding = keras.layers.Embedding(input_dim = user_num,
                                        output_dim = latent_dim,
                                        name = "user_embedding",
                                        embeddings_initalizer = "uniform",
                                        embeddings_regularizer = 12 * regs[0],
                                        input_length = 1)

item_embedding = keras.layers.Embedding(input_dim = item_num,
                                        output_dim = latent_dim,
                                        name = "item_embedding",
                                        embeddings_initalizer = "uniform",
                                        embeddings_regularizer = 12 * regs[1],
                                        input_length = 1)

user_latent = keras.layers.Flatten()(user_embedding(user_input))
item_latent = keras.layers.Flatten()(item_embedding(item_input))

element_wise_product = keras.layers.multiply([user_latent, item_latent])

prediction = keras.layers.Dense(1,
                                activation = "sigmoid",
                                kernel_initializer = "lecun_uniform",
                                name = "prediction")(element_wise_product)

model = keras.models.Model(input = [user_input, item_input],
                           output = prediction)

NameError: name 'regs' is not defined

In [12]:
keras.layers.multiply([user_input, item_input])

<tf.Tensor 'multiply/Identity:0' shape=(None, 1) dtype=int32>